# **COVID-19 by using Clustering**


# **1.Question or problem definition.**

การตั้งคำถามจากข้อมูล data set ที่ได้มา ซึ่งเป็นข้อมูล้กี่ยวกับโคโรนาไวรัส หรือชื่อทางการ COVID-19 ซึ่งมีการปพร่กระจายไปทั่วโลกตั้งแต่ปี 2019 จนถึงปัจจุบัน จากการปพร่ของเชื้อไวรัสนี้ ก่อให้เกิดผลกระทบอย่างมากต่อการใช้ชีงิตประจำวันของประชากรโลก ดังนั้นทางผู้จัดทำจึงเล็งเห็นความสำคัญของสถานการณ์นี้ จึงต้องการทำงานวิจัยหรือโปรเจคเกี่ยวกับเรื่องนี้ โดยจะดูจาก data set ที่ได้มา ต้องการทราบดังนี้
    1. Data Set มีข้อมูลอะไรบ้าง
    2. จากข้อมูลที่มีสามารถนำไปใช้ประโยชน์อะไรได้บ้าง
    3. สามารถแจงแจกเป็นภาพออกมาได้ชัดเจนได้อย่างไร
    4. อยากเห็นกลุ่มตัวเลขของผู้ติดเชื้อ การรักษาหาย และการเสียชีวิตว่ามีการจัดกลุ่มอย่างไร


# 2. Acquire data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Import ไลบาที่จำในการเรียกใช้งาน

In [ ]:
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering, FeatureAgglomeration
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.metrics as sm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder,StandardScaler, normalize
from sklearn.impute import SimpleImputer
from scipy.cluster.hierarchy import dendrogram, cophenet
from scipy.cluster.hierarchy import linkage , fcluster
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import pyplot as plt
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import pairwise_distances_argmin
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.mixture import GaussianMixture

# for Box-Cox Transformation
from scipy import stats
from mlxtend.preprocessing import minmax_scaling

import seaborn as seabornInstance 
from pylab import rcParams
import seaborn as sns
from collections import OrderedDict
cmaps = OrderedDict()
#sns.set(style="darkgrid")
import random
import glob
import copy


In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import scale

In [ ]:
data = pd.read_csv('../input/covid19-demographic-predictors/covid19_by_country.csv')
data_2 = data.copy()

In [ ]:
data.columns.values

เช็ค column ว่ามีข้อมูลอะไรบ้าง

In [ ]:
#show data
data.info()


เช็คชนิดของข้อมูลว่าเป็นชนิดอะไร

In [ ]:
data.shape

ดู shape ของมูลว่ามีขนาดเท่าไหร่

In [ ]:
data.ndim

#note ‘ndim’ which gives the number of dimensions in your data, typically 2

# **3.Wrangle, prepare, cleanse the data**

แบ่งข้อมูลเพื่อทำการ Test และ Train โมเดล เพื่อทำการ clustering แต่ก่อนจะทำการแบ่งข้อมูลนั้นต้องทำการ clean data ก่อน โดยจากข้อมูลที่ได้มามีหลาย column ที่เกิดการ missing ดังนั้นเราจะทำการจัดการข้อมูลตัวนั้นออก เพื่อให้สามารถโมเดลข้อมูลออกมาได้ดียิ่งขึ้น การเตรียมข้อมูลเพื่อให้พร้อมต่อการ train โมเล ซึ่งในั้นตอนนี้เราจะค่อย ๆ ทีไปละลำดับ โดยเช็ค missing ก่อนว่ามีมากน้อยเพียงใด 

In [ ]:
count_by_column = (data.sum())
print(count_by_column)

In [ ]:
missing_val_count_by_column = (data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

เช็คจำนวนข้อมมูลที่หายไป 

In [ ]:
data.replace([np.inf, - np.inf], np.nan)
data.isnull().any()


เช็คว่ามี column ไหนหายไปบ้าง เมื่อรู้ว่ามีคอลลัมน์ไหนหายไปแล้วก็ทำการจัดการกับข้อมูลในคอลลัมน์นั้น

In [ ]:
new_tests = data['Tests'].mean()
new_tests

หาค่าเฉลี่ยของ Tests เพื่อจะนำไปแทนค่าในส่วนที่หายไป

# **ฟังก์ชั่นการแทนค่า** 
เพื่อแทนค่าของข้อมูลที่หาไป โดยใช้การเช็คเงื่อนไขว่ามีการหายไปหรือไม่ ถ้ามีการหายไปให้รีเทิร์นค่าของค่าเฉลี่ยๆในคอลลัมน์ที่หายไป
แต่ ไม่สามารถทำได้ทุกคอลลัมน์เพราะด้วยเนื่องจากมีชนิดข้อมูลที่ต่างกัน

In [ ]:
def impute_annual_inc(x):
    if pd.isnull(x):
        return new_tests
    else:
        return x
    
data['Tests'] = data['Tests'].apply(impute_annual_inc)


In [ ]:
new_top = data['Test Pop'].mean()
new_top

In [ ]:
def impute_annual_inc(x):
    if pd.isnull(x):
        return new_top
    else:
        return x
    
data['Test Pop'] = data['Test Pop'].apply(impute_annual_inc)

In [ ]:
new_sex_ratio = data['Sex Ratio'].mean()
new_sex_ratio

In [ ]:
def impute_annual_inc(x):
    if pd.isnull(x):
        return new_sex_ratio
    else:
        return x
    
data['Sex Ratio'] = data['Sex Ratio'].apply(impute_annual_inc)

ในส่วนนี้ เนื่องงจากเกิด missing value มากกกว่า 80% ดังนั้นเราจึงตัดสินใจตัดข้อมูลที่มีการหายไปว่า 80% ออก โดยที่เราเช็คแล้วว่าข้อมูลที่เราตัดออกไปนั้น ไม่มีน่าจะมีผล่อการแบ่งกลุ่ม

In [ ]:
#drop ข้อมูลที่เกิด missing มากที่สุดออก

new_data = data.drop(data.columns[[5,6,7,8]], axis=True).head(10)
new_data


In [ ]:
new_data.isnull().any()

เช็คอีกรอบว่ามี missing value อยู่หรือไม่ ปรากฏว่าเราได้ทำการจัดการกับ missing value ไปแล้วเรียบร้อย สังเกตุได้จาก False ที่มีค่าเท่ากับ 0 ในการเช็ค null

In [ ]:
#หาค่าเฉลยของ column ที่เราจะใช้ในการ clustering ต่อไป
new_data.describe()

เมื่อเตรียมข้อมูลเรียบร้อยแล้ว ขั้นตอนต่อไปจะเป็นการวิเคราะห์ข้อมูลที่เราได้มาหลังจาก cleaning data แล้วเรียบร้อย

# ทำ One-Hot Encoding

เพื่อให้ตัวข้อมูลมีการแปลงบางตัวแปลให้เป็นตัวเลขได้ ช่วยในการลดการเกิด eror ให้ขึ้นตอนอื่นๆได้

In [ ]:
#Qua_data= pd.get_dummies(new_data['Quarantine'])

categories = ['Tests', 'Test Pop', 'Country', 'Quarantine', 'Schools', 'Restrictions']

get_numeric_data = pd.get_dummies(data.copy(), columns=categories, drop_first=True)

In [ ]:
get_numeric_data.head()

เช็คว่าตัวข้อมลถูกแปลงไปเป็นตัวเลขแล้วหรือยัง

# **4.Analyze, identify patterns, and explore the data**

การวิเคราะห์ข้อมูลจากที่เราทำการเตรียมมาแล้วเรียบร้อย ดูว่าความสัมพัทธ์หรือข้อมูลใน data set มีความเป็นไปอย่างไร


In [ ]:
df = data.groupby(["Country"])[['Tests', 'Country', 'Total Infected', 'Total Deaths', 'Total Recovered']].sum().reset_index()
sorted_By_Confirmed=df.sort_values('Total Infected',ascending=False)
sorted_By_Confirmed=sorted_By_Confirmed.drop_duplicates('Country')

In [ ]:
Recovered_rate=(sorted_By_Confirmed['Total Recovered']*100)/sorted_By_Confirmed['Total Infected']
Deaths_rate=(sorted_By_Confirmed['Total Deaths']*100)/sorted_By_Confirmed['Total Infected']
#cases_rate=(sorted_By_Confirmed.Confirmed*100)/world_Confirmed_Total

sorted_By_Confirmed['Active']=sorted_By_Confirmed['Total Infected']-sorted_By_Confirmed['Total Deaths']-sorted_By_Confirmed['Total Recovered']
sorted_By_Confirmed['Recovered Cases Rate %']=pd.DataFrame(Recovered_rate)
sorted_By_Confirmed['Deaths Cases Rate %']=pd.DataFrame(Deaths_rate)
#sorted_By_Confirmed['Total Cases Rate %']=pd.DataFrame(cases_rate)


print("Sorted By Total Infected Cases")
sorted_By_Confirmed.style.background_gradient(cmap="tab20b")

ดูยอดรวมการติดเชื้อ การรักษา และการเสีชีวิตจากเชื้อไวรัสนี้ ว่าแต่ละประเทศมีจำนวนเท่าไหร่ โดยอาศัย heatmap เป็นการแสดงค่าสีที่มีความสัมพันธ์กับตัวเลข
ถ้าจำนวนเลขน้อยแสดงว่ามีจำนวนน้อยใน column และ row นั้น โดยในตารางนั้นมีทั้งค่าตัวเลขที่เป็นจำนวนเต็มและทศนิยม ดังนั้นเราต้องคิดต่อว่าจะทำอย่างไรในการเล่นกับข้อมูลเหล่านี้


# Top 10 country have most  cases

In [ ]:
sns.set(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(15,15 ))

sns.barplot(x="Total Infected", y="Country", data=sorted_By_Confirmed.head(20),
            label="Total Infected", color="r")

# Plot the crashes where alcohol was involved
sns.set_color_codes("muted")
sns.barplot(x="Total Recovered", y="Country", data=sorted_By_Confirmed.head(20),
            label="Total Recovered", color="g")

sns.set_color_codes("muted")
sns.barplot(x="Total Deaths", y="Country", data=sorted_By_Confirmed.head(20),
            label="Total Deaths", color="b")

# Add a legend and informative axis label
ax.legend(ncol=2, loc="lower right", frameon=True)

กราฟนี้ภาพรวามของกลุ่มประเทศที่มีการติดเชื้อมากไปน้อยตามลำดับ ซึ่งจัดลำดับทั้งหมด 10 ประเทศ

In [ ]:
x=sorted_By_Confirmed.Country.head(10)
y=sorted_By_Confirmed.Tests
plt.rcParams['figure.figsize'] = (12, 10)
sns.barplot(x,y,order=x ,palette="tab20").set_title('Total Infected / Total Deaths / Total Recovered')  #graf çizdir (Most popular)

In [ ]:
sns.pairplot(new_data, diag_kind="kde", markers="+",
                 plot_kws=dict(s=50, edgecolor="b", linewidth=1),
                 diag_kws=dict(shade=True))

ใช้ pairplot เพื่อดูภาพรวมตัว data ว่าใช้อะไรไปบาง

# 5. **Model, predict and solve the problem**

ในขั้นตอนนี้เราจะทำการเลือกมา 2 โมเดลมาเปรียบเทียบกันซึ่ง ก็คือ Hirerachy และ Kmeans เราจะทำการหา Hirerachy ก่อน

# **Hirerachy Model**

ขั้นตอนนี้เป็นการทำโมเดลที่เรียกว่า Hirerachy Model ที่เป็นหนึ่งในวิธีการ clustering โดยเราจะเรียบเทียบโมเดลนี้กับ kmeans เพื่อดูผลการทดลอง


In [ ]:
X = new_data.iloc[:, [3,4]].values #.iloc for positional indexing
y = new_data.iloc[:,].values

In [ ]:
# Calculate the linkage: mergings
mergings = linkage(X, method='complete')

# Plot the dendrogram, using varieties as labels
dendrogram(mergings,
           labels=X,
           leaf_rotation=90,
           leaf_font_size=12,
)

plt.title("Hirerachy Clustrring method 'Complete'")
plt.xlabel('Cluster size')
plt.ylabel('number')
plt.show()

complete จะเป็นการหาจุดที่อยู่ไกลกันที่สุด แล้วใช้ Dendrogram ทำการรวมหรือแยกออกมาแสดงค่าให้เห็นภาพชัดเจนขึ้น

In [ ]:
linked = linkage(X, 'single')

plt.figure(figsize=(10, 8))
dendrogram(linked, truncate_mode='lastp', p=15, 
           leaf_rotation=45, leaf_font_size=15,
            show_contracted=True)

plt.title("Hirerachy Clustrring method 'Single'")
plt.xlabel('Cluster size')
plt.ylabel('number')

plt.show()

ดูความสัมพันธ์ของกลุ่มข้อมูล ที่เราสนใจโดยในกราฟด้านบนมี clusters อยู่ 3 clusters ซึ่งแต่ละอันนั้นจะแสดงให้เห็นว่าแต่ละ cluster ที่อยู่ใกล้กันที่สุดเพียงจุดเดียว และใช้ Dendrogram ในการแสดงค่ากราฟออกมา


In [ ]:
normalized_movements = normalize(X)

# Calculate the linkage: mergings
mergings = linkage(normalized_movements, method='complete')

# Plot the dendrogram
dendrogram(mergings,
           labels=X,
           leaf_rotation=90,
           leaf_font_size=12,
)

plt.title("Hirerachy Clustrring method 'Complete', Normalize")
plt.xlabel('Cluster size')
plt.ylabel('number')
plt.show()

ทำการ Normalize หรือปรับให้ตัวค่าข้อมูลอยู่ในบรรทัดฐานเดียวกัน โดยใช้วิธีการหาจุดที่อยู่ไกลกันมากที่สุดแล้วนำมา plot กราฟ ตามที่แสดงดังกราฟด้านบน

complete จะเป็นการหาจุดที่อยู่ไกลกันที่สุด แล้วใช้ Dendrogram ทำการรวมหรือแยกออกมาแสดงค่าให้เห็นภาพชัดเจนขึ้น

In [ ]:
#Generate hierarchical cluters
k = 2

Hclustering = FeatureAgglomeration(n_clusters=k, affinity='euclidean', linkage='ward')

In [ ]:
Hclustering.fit(X)

Hclustering.labels_

หา range ของ Hirerachy Clustering ว่ามีขนาดเท่าไหร่ เพราะสามารถนำไปหา performance ได้

In [ ]:
#plot graph

plt.figure(figsize=(10,10))
plt.title("Scatter plot")
plt.scatter(X[:,0], X[:, 1])

# **K-Means**

สำหรับส่วนนี้ ผู้จัดทำตัดสินใจในการเขียฟังก์ชั่นของ K-Means ขึ้นมาโดยเพื่อแก้ไขปัญหาการบัคของโปรแกรม เมื่อเกิดบัคจะได้ตรวจหาผิดพลาดได้

In [ ]:
#set random seed

random.seed(30)

In [ ]:
# fitting multiple k-means algorithms and storing the values in an empty list
SSE = []
for cluster in range(1,20):
    kmeans = KMeans(n_jobs = -1, n_clusters = cluster, init='k-means++')
    kmeans.fit(get_numeric_data)
    SSE.append(kmeans.inertia_)

# converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

จากตรางด้านบนนั้น ใน K-means โมเดลที่มีค่าประมาณ 4 มีความสัมพันธ์กับค่า inertia ที่ต่ำ ถือว่าดี ดังนั้นการที่เราจะทำ cluter จะใช้ทั้งหมด 4 cluters ในการจัดกลุ่มของข้อมูล

In [ ]:
model = KMeans(n_clusters = 4)
model.fit(get_numeric_data)

In [ ]:
labels = model.predict(get_numeric_data)
print(labels)

In [ ]:
km = KMeans(
    n_clusters=4, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04
)

  

y_km = km.fit_predict(X)




In [ ]:
plt.scatter(
    X[y_km == 0, 0], X[y_km == 0, 1],
    s=150, c='lightgreen',
    label='cluster 1'
)

plt.scatter(
    X[y_km == 1, 0], X[y_km == 1, 1],
    s=150, c='orange',
    label='cluster 2'
)

plt.scatter(
    X[y_km == 2, 0], X[y_km == 2, 1],
    s=150, c='lightblue',
    label='cluster 3'
)

plt.scatter(
    X[y_km == 3, 0], X[y_km == 3, 1],
    s=150, c='yellow',
    label='cluster 4'
)




# plot the centroids
plt.scatter(
    km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
    s=15,
    c='red',label='centroids'
)


#plt.legend(scatterpoints=1)
plt.legend()
plt.title('Visualization of clustered data', fontweight='bold')
ax.set_aspect('equal');
plt.show()

 plot กราฟ เพื่อแสดงกลุ่มของข้อมูลที่เราได้ทำการจัดกลุ่มจาก dataset
 ซึ่งจากกราฟที่ได้นั้นเป็นกราฟยังไม่สมบูรณ์ การทำ cluster ยังมีข้อผิดพลาด ดังนั้นต้องจัดการกับข้อมูลใหม่ตามวิธีด้านล่าง

# แก้ไขข้อมูลของ K-Mean 

โดยการ Visualize, Normalize และ Scaling data set ให้อยู่ใน range เดียวกัน ทำเป็น step by step ดังนี้

#  TNSE

TSNE จะเป็น tool ในการแปลงค่าของตัวแปลที่มีการทำ one-hot edcoding  ให้กลับไปอยู่ใน range เพื่อไม่ให้ข้อมูลเกิดการกระจ่ายตัวกันเกินไป

In [ ]:
model = TSNE(learning_rate=200)

# Apply fit_transform to samples: tsne_features
transformed = model.fit_transform(get_numeric_data)


In [ ]:
transformed[1:1,:]

In [ ]:
get_numeric_data.columns

set ว่าจะให้ไว้กี่มิติ ในที่นี้เซตเป็น 1x1 มิติ

In [ ]:
get_numeric_data.shape

In [ ]:
get_numeric_data['x'] = transformed[:,0]
get_numeric_data['y'] = transformed[:,1]

In [ ]:
x = get_numeric_data.iloc[:, [3,4]].values #.iloc for positional indexing
y = get_numeric_data.iloc[:,].values

In [ ]:
sns.scatterplot(get_numeric_data['x'] , get_numeric_data['y'] , data=get_numeric_data)

plt.title('After did TSNE', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.show()

กราฟนี่แสดงผลลัพธ์ของการ Normalized data โดยการทำเป็น TSNE เพื่อแหลงค่าตัวแปล get_numeric_data ให้กละบมาอยู่ใน range เดิม โดยผลลัพธ์จะเป็นกราฟสุ่มออกเป็นภ่าพที่ไม่ซ้ำ

 # 6. Visualize, report, and present the problem solving steps and final solution

# Re-do K-MEAN again

In [ ]:
#set random seed
np.random.seed(200)

In [ ]:
# fitting multiple k-means algorithms and storing the values in an empty list
SSE = []
for cluster in range(1,20):
    kmeans = KMeans(n_jobs = -1, n_clusters = cluster, init='k-means++')
    kmeans.fit(transformed)
    SSE.append(kmeans.inertia_)

# converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

กราฟนี้เรียกว่า elbow เป็นการสร้างเพื่อดูผลัพธ์แล้วนำไปวิเคราะห์ว่าจะใช้จำนวน cluster เท่าไหร่ ซึ่งดูจากกราฟที่อยู่เกือบจะที่มีค่าที่เสถียรสุด ในโปรเจคนี้เราได้เลือกจำนวน cluster = 5

In [ ]:
k = 5  #ประการตัวแปร

In [ ]:
x = get_numeric_data.iloc[:, [3,4]].values #.iloc for positional indexing
y = get_numeric_data.iloc[:,].values

In [ ]:
# centroids[i] = [x, y]
centroids = {
    i+1: [np.random.randint(-1, 80), np.random.randint(-1, 80)]
    for i in range(k)
}
    
fig = plt.figure(figsize=(10, 10))
plt.scatter(['x'],['y'], color='#5742FD')
colmap = {1: 'r', 2: 'g', 3: 'b', 4: 'cyan', 5: 'yellow'}
for i in centroids.keys():
    plt.scatter(*centroids[i], color=colmap[i])
    
plt.title('Visualization of clustered data', fontweight='bold')
#ax.set_aspect('equal');

plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.show()

In [ ]:
def assignment(df, centroids):
    for i in centroids.keys():
        # sqrt((x1 - x2)^2 - (y1 - y2)^2)
        df['distance_from_{}'.format(i)] = (
            np.sqrt(
                (get_numeric_data['x'] - centroids[i][0]) ** 2
                + (get_numeric_data['y'] - centroids[i][1]) ** 2
            )
        )
    centroid_distance_cols = ['distance_from_{}'.format(i) for i in centroids.keys()]
    df['closest'] = df.loc[:, centroid_distance_cols].idxmin(axis=1)
    df['closest'] = df['closest'].map(lambda x: int(x.lstrip('distance_from_')))
    df['color'] = df['closest'].map(lambda x: colmap[x])
    return df

#print(df.head())

In [ ]:
df = assignment(get_numeric_data, centroids)


In [ ]:
fig = plt.figure(figsize=(10,10))
plt.scatter(df['x'], df['x'], color=df['color'], alpha=0.5, edgecolor='k')
for i in centroids.keys():
    plt.scatter(*centroids[i], color=colmap[i])
    
#plt.xlim(0, 80)
#plt.ylim(0, 80)
plt.title('K-Means (get_numeric_data)', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.show()

เช็คหาจุดของข้อมูลที่อยู่ในกราฟ โดยตัว centriods จะหาจุดึ่งของตัว cluster แล้วเมพไปเรื่อยๆ จนกราฟจุดนิ่ง แล้วก็จะแสดงผลลัพธ์สุดท้ายออกมา

In [ ]:
old_centroids = copy.deepcopy(centroids)

def update(k):
    for i in centroids.keys():
        centroids[i][0] = np.mean(df[df['closest'] == i]['x'])
        centroids[i][1] = np.mean(df[df['closest'] == i]['y'])
    return k

centroids = update(centroids)
    
fig = plt.figure(figsize=(10,10))
ax = plt.axes()
plt.scatter(df['x'], df['y'], color=df['color'], alpha=0.5, edgecolor='k')
for i in centroids.keys():
    plt.scatter(*centroids[i], color=colmap[i])
#plt.xlim(0, 80)
#plt.ylim(0, 80)
for i in old_centroids.keys():
    old_x = old_centroids[i][0]
    old_y = old_centroids[i][1]
    dx = (centroids[i][0] - old_centroids[i][0]) * 0.75
    dy = (centroids[i][1] - old_centroids[i][1]) * 0.75
    ax.arrow(old_x, old_y, dx, dy, head_width=0.5, head_length=0.5, fc=colmap[i], ec=colmap[i])

plt.title('K-Means (get_numeric_data)', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.show()

สังเกตได้วว่า centriods ทัการเปลี่ยนแปลงไป เพื่อให้เข้าใกล้กลุ่มข้อมูล

In [ ]:
df = assignment(df, centroids)

# Plot results
fig = plt.figure(figsize=(10,10))
plt.scatter(df['x'], df['y'], color=df['color'],alpha=0.5, edgecolor='k')
for i in centroids.keys():
    plt.scatter(*centroids[i], color=colmap[i])
#plt.xlim(0, 80)
#plt.ylim(0, 80)
plt.title('K-Means (get_numeric_data)', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.show()

เมื่อจุก centriods คงที่แล้ว จะเห็นได้ว่า scatter plot มีการจัดกลุ่มได้ชัดเจนและเป็นระเบียบมากขึ้น ทำให้ง่ายต่อการดูกราฟ

In [ ]:
# Continue until all assigned categories don't change any more
while True:
    closest_centroids = df['closest'].copy(deep=True)
    centroids = update(centroids)
    df = assignment(df, centroids)
    if closest_centroids.equals(df['closest']):
        break

fig = plt.figure(figsize=(10,10))
plt.scatter(df['x'], df['y'], color=df['color'], alpha=0.5, edgecolor='k')
for i in centroids.keys():
    plt.scatter(*centroids[i], color=colmap[i])
#plt.xlim(0, 80)
#plt.ylim(0, 80)
plt.title('K-Means (get_numeric_data)', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.show()


นี่คือผลลัพธ์สุดท้ายของการทำ k-Mean Clustering และจะเห็นว่า centriods กับตัวกลุ่มข้อมูลนั้นมาค่าใกล้เคียงกันมาที่สุดในแต่ละ clusters

In [ ]:
get_numeric_data

หลังจากทำการหา k-mean ข้างต้นไปนั้น เราจะมาแสดงผลของค่า get_numeric_data อีกครั้งเพื่อดูว่าอะไรเปลี่ยนแปลง โดยจะเห็นว่ามี columns เพิ่มขึ้นมาหลายคอลัม นี่คือเป็นผลลัพธ์ของการทำ TSNE และการหา K-Means clustering นั้นเอง

In [ ]:
new_data2 = get_numeric_data.drop(get_numeric_data.columns[[246]], axis=True).head(10)
new_data2

เพื่องด้วยเราต้องการเปรียบเทียบตัว get_numeric_data กับตัว new_data2 เราจึงทำหาร drop คอลัมที่ 246 ออกเพราะไม่จำเป็นและทำให้เกิด error ในโปรแกรม

In [ ]:
new_data2.replace([np.inf, - np.inf], np.nan)
new_data2.isnull().any()

จากนั้นเช็คดูว่ามี missing value ใน new_data2  หรือไม่ เหมือนเป็นการทำซ้ำจากส่วนที่ผ่านมา แต่เป็นข้อมูลอีกเซตที่เราได้ทำการเตรียมเอาไว้แล้ว

In [ ]:
new_data2.info()

ดูว่ามีตัวแปรชนิดไหนบาง เพือที่จะสามารถทำ model ได้

In [ ]:
X = new_data2.iloc[:, [3,4]].values #.iloc for positional indexing
y = new_data2.iloc[:,].values

เซตค่า X และ x เอาไว้ เพื่อให้โปรแกรมดึงไปใช้งาน โดยให้อยู่ใน range เดียวกัน

In [ ]:
km = KMeans(
    n_clusters=5, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04
)

  

y_km = km.fit_predict(X)


เซตค่าข้อมูลที่เราจะทำการโมเดลออกมาว่าอยากให้มัจำนวน cluster เท่าไหร่ แล้วทำการ fit โมเดลนั้น

In [ ]:
plt.scatter(
    X[y_km == 0, 0], X[y_km == 0, 1],
    s=150, c='lightgreen',
    label='cluster 1'
)

plt.scatter(
    X[y_km == 1, 0], X[y_km == 1, 1],
    s=150, c='orange',
    label='cluster 2'
)

plt.scatter(
    X[y_km == 2, 0], X[y_km == 2, 1],
    s=150, c='lightblue',
    label='cluster 3'
)

plt.scatter(
    X[y_km == 3, 0], X[y_km == 3, 1],
    s=150, c='yellow',
    label='cluster 4'
)

plt.scatter(
    X[y_km == 4, 0], X[y_km == 4, 1],
    s=150, c='cyan',
    label='cluster 5'
)


# plot the centroids
plt.scatter(
    km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
    s=15,
    c='red',label='centroids'
)


#plt.legend(scatterpoints=1)
plt.legend()
plt.title('After Normalization data', fontweight='bold')
ax.set_aspect('equal');
plt.show()

กราฟด้านบนการจัดกลุ่มของข้อมูลหลังจากทำการจัดการชุดข้อมูลใหม่ ให้มีการจัก range และ scale ให้ใกล้เคียงกัน ผลลัพธ์ออกมาจึงสามารถดูง่ายและจัดเรียงได้จัดเจน

**เพิ่มเติม** 
ลองใช้ใน fucntion K-Mean อีกรูปแบบที่เป็น โค้ดแบบเต็มว่าจะมีความแกต่างกันหรือไม่

In [ ]:
x = new_data2.iloc[:, [3,4]].values #.iloc for positional indexing
y = new_data2.iloc[:,].values

In [ ]:
k = 5

# centroids[i] = [x, y]
centroids = {
    i+1: [np.random.randint(-1, 80), np.random.randint(-1, 80)]
    for i in range(k)
}
    
fig = plt.figure(figsize=(10, 10))
plt.scatter(['x'],['y'], color='#5742FD')
colmap = {1: 'r', 2: 'g', 3: 'b', 4: 'cyan', 5: 'yellow'}
for i in centroids.keys():
    plt.scatter(*centroids[i], color=colmap[i])
    
plt.title('Visualization of clustered data', fontweight='bold')
#ax.set_aspect('equal');

plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.show()

In [ ]:
def assignment(df, centroids):
    for i in centroids.keys():
        # sqrt((x1 - x2)^2 - (y1 - y2)^2)
        df['distance_from_{}'.format(i)] = (
            np.sqrt(
                (get_numeric_data['x'] - centroids[i][0]) ** 2
                + (get_numeric_data['y'] - centroids[i][1]) ** 2
            )
        )
    centroid_distance_cols = ['distance_from_{}'.format(i) for i in centroids.keys()]
    df['closest'] = df.loc[:, centroid_distance_cols].idxmin(axis=1)
    df['closest'] = df['closest'].map(lambda x: int(x.lstrip('distance_from_')))
    df['color'] = df['closest'].map(lambda x: colmap[x])
    return df

#print(df.head())

In [ ]:
df = assignment(new_data2, centroids)

fig = plt.figure(figsize=(10,10))
plt.scatter(df['x'], df['x'], color=df['color'], alpha=0.5, edgecolor='k')
for i in centroids.keys():
    plt.scatter(*centroids[i], color=colmap[i])
#plt.xlim(0, 80)
#plt.ylim(0, 80)
plt.title('K-Means (new_data2)', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.scatter(df['x'], df['x'], color=df['color'], alpha=0.5, edgecolor='k')
for i in centroids.keys():
    plt.scatter(*centroids[i], color=colmap[i])
#plt.xlim(0, 80)
#plt.ylim(0, 80)
plt.title('K-Means (new_data2)', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.show()

In [ ]:
old_centroids = copy.deepcopy(centroids)

def update(k):
    for i in centroids.keys():
        centroids[i][0] = np.mean(df[df['closest'] == i]['x'])
        centroids[i][1] = np.mean(df[df['closest'] == i]['y'])
    return k

centroids = update(centroids)
    
fig = plt.figure(figsize=(10,10))
ax = plt.axes()
plt.scatter(df['x'], df['y'], color=df['color'], alpha=0.5, edgecolor='k')
for i in centroids.keys():
    plt.scatter(*centroids[i], color=colmap[i])
#plt.xlim(0, 80)
#plt.ylim(0, 80)
for i in old_centroids.keys():
    old_x = old_centroids[i][0]
    old_y = old_centroids[i][1]
    dx = (centroids[i][0] - old_centroids[i][0]) * 0.75
    dy = (centroids[i][1] - old_centroids[i][1]) * 0.75
    ax.arrow(old_x, old_y, dx, dy, head_width=0.5, head_length=0.5, fc=colmap[i], ec=colmap[i])
    
plt.title('K-Means (new_data2)', fontsize=18)    
plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.show()

In [ ]:
df = assignment(df, centroids)

# Plot results
fig = plt.figure(figsize=(10,10))
plt.scatter(df['x'], df['y'], color=df['color'],alpha=0.5, edgecolor='k')
for i in centroids.keys():
    plt.scatter(*centroids[i], color=colmap[i])
#plt.xlim(0, 80)
#plt.ylim(0, 80)
plt.title('K-Means (new_data2)', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.show()

In [ ]:
# Continue until all assigned categories don't change any more
while True:
    closest_centroids = df['closest'].copy(deep=True)
    centroids = update(centroids)
    df = assignment(df, centroids)
    if closest_centroids.equals(df['closest']):
        break

fig = plt.figure(figsize=(10,10))
plt.scatter(df['x'], df['y'], color=df['color'], alpha=0.5, edgecolor='k')
for i in centroids.keys():
    plt.scatter(*centroids[i], color=colmap[i])
#plt.xlim(0, 80)
#plt.ylim(0, 80)
plt.title('K-Means (new_data2)', fontsize=18)
plt.xlabel('x', fontsize=18)
plt.ylabel('y', fontsize=18)
plt.show()

In [ ]:
km = KMeans(
    n_clusters=5, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04
)

  

y_km = km.fit_predict(x)

In [ ]:
plt.scatter(
    x[y_km == 0, 0], x[y_km == 0, 1],
    s=150, c='lightgreen',
    label='cluster 1'
)

plt.scatter(
    x[y_km == 1, 0], x[y_km == 1, 1],
    s=150, c='orange',
    label='cluster 2'
)

plt.scatter(
    x[y_km == 2, 0], x[y_km == 2, 1],
    s=150, c='lightblue',
    label='cluster 3'
)

plt.scatter(
    X[y_km == 3, 0], X[y_km == 3, 1],
    s=150, c='yellow',
    label='cluster 4'
)

plt.scatter(
    x[y_km == 4, 0], x[y_km == 4, 1],
    s=150, c='cyan',
    label='cluster 5'
)


# plot the centroids
plt.scatter(
    km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
    s=15,
    c='red',label='centroids'
)


#plt.legend(scatterpoints=1)
plt.legend()
plt.title('After Normalization data', fontweight='bold')
ax.set_aspect('equal');
plt.show()

# สรุป

 หลังจากที่ทำกระบวนการทั้ง 5 กระบวนการ จนได้ผลลัพธ์ทั้งหมด ซึ่งสามารถเปรียเทียบการทำงานของแต่ละโมเดลได้อย่างชัดเจนคือ
     1. Hierarchy
        - สามารถนำชุดของมูลหลังจากจัดการ missing value มาใช้ได้เลย
        - ไม่มีความซับซ้อนในการทำงาน หรือสร้างกราฟ
        - กราฟที่ได้ในบ้าง method ดูยาก และยังไม่สามารถนำไปใช้งานหรืออธิบายได้
        - เมื่อมีการเพิ่ม normalzed เข้าไปด้วยทำให้กราฟดูชัดขึ้น แต่การจับกลุ่มยังดูเข้าใจยาก

     2. K-Mean 
        - วิธีการซับซ้อนกว่า Hierarchy 
        - วิธีการเตรียมข้อมูลเยอะ แต่มีการปรับ scale และ range ของตัวแปรให้ใกล้เคียงกัน
        - เมื่อสร้างกราฟออกมาเกิดภาพการจับกลุ่มที่ดูง่ายแบ่งสีชัดเจน
        - output ของ cluster ที่ออกมามีความเข้าใจง่าย สีสวยงาม และสามารถนำไปวิเคาระห์ต่อได้ง่ายขึ้น
        
    
 ความสัมพันธ์ของข้อมูลเมื่อมีการจัดกลุ่มหรือจับคู่ สามารถนำไปวิเคาระห์เพื่อต่อยอดข้อมูลชุดนี้ในด้านต่างๆเช่น การแพทย์ การคมนาคม การพัฒนาบ้านเมือง เป็นต้น โดยการจัดทำ modeling ข้อมูลชุดนี้ทางผู้จัดทำหวังเป็นอย่างยิ่งว่าจะสามารถสร้างประโยชน์ต่อสาธารณะไม่มากก็น้อย
         